# **7. Gradient Boosting**

In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
def xgboost_classification(X_train, y_train, X_test, y_test, mode, save_file_name):

    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.2, random_state=42)

    # define the parameter grid to search over
    param_grid = {
        'learning_rate': [0.01, 0.1, 1.0],
        'max_depth': [3, 5, 7, 9, 11],
        'n_estimators': [50, 100, 150, 200, 250],
        'min_child_weight': [1, 3, 5, 7, 9]
    }

    # create an XGBoost classifier object
    xgb_model = xgb.XGBClassifier(tree_method="gpu_hist", gpu_id=0)

    # create a GridSearchCV object
    if mode == "BINARY":
        grid_search = GridSearchCV(
            xgb_model, param_grid, scoring='f1', cv=7, n_jobs=-1, refit=True)
    elif mode == "MULTICLASS":
        grid_search = GridSearchCV(
            xgb_model, param_grid, scoring='f1_micro', cv=7, n_jobs=-1, refit=True)

    # fit the GridSearchCV object to the data
    grid_search.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=0)

    # print the best hyperparameters and score
    print("Best hyperparameters: ", grid_search.best_params_)
    print("Best score: ", grid_search.best_score_)

    # Use the model to make predictions on the test data
    y_pred = grid_search.predict(X_test)

    # Get the precision and recall scores
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    print("Precision:", precision)
    print("Recall:", recall)

    # Get the F1 score
    # Use 'weighted' if you have imbalanced classes
    f1 = f1_score(y_test, y_pred, average='weighted')
    print("F1 Score:", f1)

    # Get the classification report
    report = classification_report(y_test, y_pred)
    print("Classification Report:\n", report)

## **White**

### 1) White Win/Loss Classification

In [ ]:
X_train = pd.read_csv("/notebooks/train-test-split/white-win-loss/wwcX_train.csv")
X_test = pd.read_csv("/notebooks/train-test-split/white-win-loss/wwcX_test.csv")
y_train = pd.read_csv("/notebooks/train-test-split/white-win-loss/wwcY_train.csv")
y_test = pd.read_csv("/notebooks/train-test-split/white-win-loss/wwcY_test.csv")

In [ ]:
# define the parameter grid to search over
param_grid = {
    'learning_rate': [0.01, 0.1, 1.0],
    'max_depth': [3, 5, 7, 9, 11],
    'n_estimators': [50, 100, 150, 200, 250],
    'min_child_weight': [1, 3, 5, 7, 9]
}

# create an XGBoost classifier object
xgb_model = xgb.XGBClassifier(tree_method='gpu_hist', predictor='gpu_predictor')

# create a GridSearchCV object
grid_search = GridSearchCV(xgb_model, param_grid, scoring='f1', cv=5, verbose=1, n_jobs=-1, refit=True)

# fit the GridSearchCV object to the data
grid_search.fit(X_train, y_train)

# print the best hyperparameters and score
print("Best hyperparameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

# Use the model to make predictions on the test data
y_pred = grid_search.predict(X_test)

# Get the accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Get the classification report
report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)

Fitting 5 folds for each of 375 candidates, totalling 1875 fits
Best hyperparameters:  {'learning_rate': 1.0, 'max_depth': 11, 'min_child_weight': 1, 'n_estimators': 250}
Best score:  0.7378939788141305
Accuracy: 0.7496281728228136
Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.74      0.74     79792
           1       0.75      0.76      0.75     82918

    accuracy                           0.75    162710
   macro avg       0.75      0.75      0.75    162710
weighted avg       0.75      0.75      0.75    162710



In [ ]:
xgb_model_demo = xgb.XGBClassifier(
    n_estimators=350, 
    max_depth=15, 
    learning_rate=1.0, 
    min_child_weight=1
  )

xgb_model_demo.fit(X_train, y_train)
y_pred_demo = xgb_model_demo.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_demo)
print("Accuracy:", accuracy)

report_demo = classification_report(y_test, y_pred_demo)
print("Classification Report:\n", report_demo)

Accuracy: 0.7990535308217074
Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.79      0.79     79792
           1       0.80      0.80      0.80     82918

    accuracy                           0.80    162710
   macro avg       0.80      0.80      0.80    162710
weighted avg       0.80      0.80      0.80    162710



### 2) White Termination Classification

In [ ]:
### White Termination Classification ###
print("Starting White Termination Classification...")

wtcX_train = pd.read_csv('./train-test-split/white-termination/wtcX_train.csv')
wtcX_test = pd.read_csv('./train-test-split/white-termination/wtcX_test.csv')
wtcY_train = pd.read_csv('./train-test-split/white-termination/wtcY_train.csv')
wtcY_test = pd.read_csv('./train-test-split/white-termination/wtcY_test.csv')

xgboost_classification(
    wtcX_train, wtcY_train['Termination'], wtcX_test, wtcY_test['Termination'], "MULTICLASS", "wtc")

<img src="./gb-results/white-termination.png">

## **Black**

### 1) Black Win/Loss Classification

In [ ]:
# define the parameter grid to search over
param_grid = {
    'learning_rate': [0.01, 0.1, 1.0],
    'max_depth': [3, 5, 7, 9, 11],
    'n_estimators': [50, 100, 150, 200, 250],
    'min_child_weight': [1, 3, 5, 7, 9]
}

# create an XGBoost classifier object
xgb_model = xgb.XGBClassifier(tree_method='gpu_hist', predictor='gpu_predictor')

# create a GridSearchCV object
grid_search = GridSearchCV(xgb_model, param_grid, scoring='f1', cv=5, verbose=1, n_jobs=-1, refit=True)

# fit the GridSearchCV object to the data
grid_search.fit(X_train, y_train)

# print the best hyperparameters and score
print("Best hyperparameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

# Use the model to make predictions on the test data
y_pred = grid_search.predict(X_test)

# Get the accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Get the classification report
report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)

Fitting 5 folds for each of 375 candidates, totalling 1875 fits
Best hyperparameters:  {'learning_rate': 1.0, 'max_depth': 11, 'min_child_weight': 1, 'n_estimators': 250}
Best score:  0.7208037263480291
Accuracy: 0.7467088685391187
Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.76      0.75     83059
           1       0.74      0.74      0.74     79651

    accuracy                           0.75    162710
   macro avg       0.75      0.75      0.75    162710
weighted avg       0.75      0.75      0.75    162710



In [ ]:
xgb_model_demo = xgb.XGBClassifier(
    n_estimators=350, 
    max_depth=15, 
    learning_rate=1.0, 
    min_child_weight=1
  )

xgb_model_demo.fit(X_train, y_train)
y_pred_demo = xgb_model_demo.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_demo)
print("Accuracy:", accuracy)

report_demo = classification_report(y_test, y_pred_demo)
print("Classification Report:\n", report_demo)

Accuracy: 0.7957716182164587
Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.80      0.80     83059
           1       0.79      0.79      0.79     79651

    accuracy                           0.80    162710
   macro avg       0.80      0.80      0.80    162710
weighted avg       0.80      0.80      0.80    162710



### 2) Black Termination Classification

In [ ]:
### Black Termination Classification ###
print("Starting Black Termination Classification...")

btcX_train = pd.read_csv('./train-test-split/black-termination/btcX_train.csv')
btcX_test = pd.read_csv('./train-test-split/black-termination/btcX_test.csv')
btcY_train = pd.read_csv('./train-test-split/black-termination/btcY_train.csv')
btcY_test = pd.read_csv('./train-test-split/black-termination/btcY_test.csv')

xgboost_classification(
    btcX_train, btcY_train['Termination'], btcX_test, btcY_test['Termination'], "MULTICLASS", "btc")

<img src="./gb-results/black-termination.png">